## Trees


#### The Closure Property of Data Types
The ability to use lists as the elements of other lists provides a new means of combination in our programming language.
- A method for combining data values satisfies the closure property if:    
  The result of combination can itself be combined using the same method
- Closure is  powerful as it permits us to create hierarchical structures.

### Box-and-Pointer Notation
Box-and-pointer notation is a way to reprsent lists within our environment diagrams.    
We need the notation as sequencial data can be complicated due to their closure property.

### Box-and-Pointer Notation in Environment Diagrams
Lists are represented as a row of index-labeled adjacent boxes, one per element.   
Each box either containes a primitive value or points to a compound value by an arrow.
![image.png](attachment:image.png)

### Tree Abstraction

The tree is a fundamental data abstraction that imposes regularity of how hierarchical values are structured and manipulated.


#### Tree metaphors:
1. Recursive description(wooden trees):
  - A tree has a root label and a list of branches
  - Each branch is a tree
  - A tree with zero branches is called sleaf
2. Relativa description(family trees):
  - Each location in a tree is called a node
  - Each node has a label that can be any value.
  - One node can be the parent/child of another

People often refer to labels by their locations:
e.g. each parent is the sum of its children

#### Implementing the tree abstraction
The data abstraction for a tree consists of the constructor `tree` and the selectors `label` and `branches`.

- A root label and a *list* of branches
- Each branch is a tree

In [28]:
def tree(root_label, branches=[]):
    for branch in branches:
        assert is_tree(branch), "branches must be trees"
    # check branches are trees
    return [root_label] + list(branches)# in case of passing other sequences


def label(tree):
    return tree[0]

def branches(tree):
    return tree[1:]

def is_tree(tree):
    if type(tree) != list or len(tree) < 1:
        return False
    for branch in branches(tree):
        if not is_tree(branch):
            return False
    #Note: branches 为[]时不会进行循环
    return True

def is_leaf(tree):
    return not branches(tree)

In [29]:
t = tree(3, [tree(1), tree(2, [tree(1), tree(1)])])
t

[3, [1], [2, [1], [1]]]

In [30]:
label(t)

3

In [31]:
branches(t)

[[1], [2, [1], [1]]]

In [32]:
label(branches(t)[0])
# treating branches as a list is not violating abstraction barrier
# because in our abstraction, a tree has a list of branches 

1

In [33]:
is_leaf(branches(t)[0])

True

Tree-recursive functions can be used to construct trees.

In [34]:
def fib_tree(n):
    if n == 0 or n == 1:
        return tree(n)
    else:
        left, right = fib_tree(n-2), fib_tree(n-1)
        fib_n = label(left) + label(right)
        return tree(fib_n, [left, right])


In [35]:
fib_tree(5)

[5, [2, [1], [1, [0], [1]]], [3, [1, [0], [1]], [2, [1], [1, [0], [1]]]]]

### Tree Processing
Functions that take trees as input or return trees as output are often tree recursive themselves.      
- Processing a leaf is often the base case of a tree processing function.   
- The recursive case typically makes a recursive call on each branch,then aggregates.


In [36]:
def count_leaves(tree):
    if is_leaf(tree):
        return 1
    else:
        branch_counts = [count_leaves(b) for b in branches(tree)]
        return sum(branch_counts)

In [37]:
count_leaves(fib_tree(5))

8

In [40]:
def leaves(tree):
    if is_leaf(tree):
        return [label(tree)]
    #不直接返回tree是因为希望将非列表转化为列表形式
    else:
        l = sum([leaves(l) for l in branches(tree)],[])
        return l

In [41]:
leaves(fib_tree(5))

[1, 0, 1, 0, 1, 1, 0, 1]

- Creating Trees   
A function that creates a tree from anthoer tree is typically also recursive.


In [50]:
def increment_leaves(t):
    """Return a tree like t but with leaf labels incremented"""
    if is_leaf(t):
        return tree(label(t)+1)
    else:
        bs = [increment_leaves(b) for b in branches(t)]
        return tree(label(t), bs)
def increment(t):
    """Return a tree like t but with all labels incremented"""
    if is_leaf(t):
        return tree(label(t)+1)
    else:
        bs = [increment(b) for b in branches(t)]
        return tree(label(t)+1, bs)

We dont have to set a base case if every tree is treated the same way, as the list comprehension has accounted for the [] case.

In [52]:
def increment(t):
    """Return a tree like t but with all labels incremented"""
    return tree(label(t)+1, [increment(b) for b in branches(t)])

In [53]:
increment(fib_tree(5))

[6, [3, [2], [2, [1], [2]]], [4, [2, [1], [2]], [3, [2], [2, [1], [2]]]]]

In [49]:
increment_leaves(fib_tree(5))

[5, [2, [2], [1, [1], [2]]], [3, [1, [1], [2]], [2, [2], [1, [1], [2]]]]]

In [56]:
def print_tree(t, indent=0):
    print(' ' * indent + str(label(t)))
    for b in branches(t):
        print_tree(b, indent+1)

In [57]:
print_tree(fib_tree(5))

5
 2
  1
  1
   0
   1
 3
  1
   0
   1
  2
   1
   1
    0
    1


#### Example: Summing Paths

Recursive functions can build up their results by:
- manipulating the return value of a recursive call



In [58]:
def fact(n):
    if n == 0:
        return 1
    else:
        return n * fact(n-1)

- passing information into the recursive call as an argument    
The base case needs to be the same result as the original recursive call.    
And when we reach the base case the operation has been done already.

In [ ]:
def fact_times(n, k):
    """Return k*n*(n-1)..."""
    if n == 0:
        return k
    else:
        return fact_times(n-1, k*n)

In [59]:
def fact(n):
    return fact_times(n, 1)

In [60]:
def print_sums(t, so_far):
    if is_leaf(t):
        print(so_far)
    else:
        for b in branches(t):
            print_sums(b, so_far+label(t))


#### Example: Count paths

In [61]:
def count_paths(t, total):
    if label(t) == total:
        found = 1
    else:
        found = 0
    return found + sum([count_paths(b, total-label(t)) for  b in branches(t)])